In [ ]:
# !pip install tensorflow tbparse
# %pip install tbparse
from datetime import datetime
import time as time
import math
from statistics import mean
import json as json
import numpy as np
from tbparse import SummaryReader
import pandas as pd
from matplotlib import pyplot as pyplot

## LSTM-PPO, PPO, DRQN Training Data

In [ ]:
# training data
log_dir = './${PPO_LOG_DIR}'
reader = SummaryReader(log_dir, pivot=True)
# print(reader.get_tags())
df_ppo = reader.scalars


log_dir = './${RPPO_LOG_DIR}'
reader = SummaryReader(log_dir, pivot=True)
# print(reader.get_tags())
df_rppo = reader.scalars

In [ ]:
# training data ------- environment scalars
log_dir = './${PPO_LOG_DIR}/scalars'
reader = SummaryReader(log_dir, pivot=True)
# print(reader.get_tags())
df_ppo_ = reader.tensors


log_dir = './${RPPO_LOG_DIR}/scalars'
reader = SummaryReader(log_dir, pivot=True)
# print(reader.get_tags())
df_rppo_ = reader.tensors

log_dir = './${DQN_LOG_DIR}'
reader = SummaryReader(log_dir, pivot=True)
# print(reader.get_tags())
df_dqn_ = reader.tensors

In [ ]:
# agents trained for 5 minutes for motre than 500 episodes
ppox = df_ppo_['step'].to_numpy()[:533]
ppoy = [i for i in df_ppo_[df_ppo_['episode'] <= 533]['mean_reward'] if not math.isnan(i)]

rppox = df_rppo_['step'].to_numpy()[:533]
rppoy = [i for i in df_rppo_[df_rppo_['episode'] <= 533]['mean_reward'] if not math.isnan(i)][:533]

dqn_x = [i for i in range(533)]
dqn_y = [i for i in df_dqn_[df_dqn_['episode'] <= 533]['mean_reward'] if not math.isnan(i)][:533]
pyplot.plot(ppox, ppoy, color='red')
pyplot.plot(rppox, rppoy, color='green')
pyplot.plot(dqn_x, dqn_y, color='blue')
pyplot.xlabel('Episodes (5 minutes)')
pyplot.ylabel('Mean Episodic Reward')
pyplot.title('MatMul/Training')
pyplot.legend(['PPO', 'RPPO', 'DRQN'], loc='lower right')
pyplot.show()

In [ ]:
# In case we want the Utilization plot
# CPU Utilization
ppox = df_ppo_['step'].to_numpy()
ppoy = df_ppo_['cpu'].to_numpy()
ppoy = df_ppo_['cpu'].rolling(window=500, min_periods=10).mean().to_numpy()
rppox = df_rppo_['step'].to_numpy()
rppoy = df_rppo_['cpu'].to_numpy()
rppoy = df_rppo_['cpu'].rolling(window=500, min_periods=10).mean().to_numpy()
dqn_x = df_dqn_['step'].to_numpy()
dqn_y = df_dqn_['cpu'].to_numpy()
dqn_y = df_dqn_['cpu'].rolling(window=500, min_periods=10).mean().to_numpy()

pyplot.plot(ppox, ppoy, color='red')
pyplot.plot(rppox[:7705], rppoy[:7705], color='green')
pyplot.plot(dqn_x[:7170], dqn_y[:7170], color='blue')
pyplot.xlabel('Sampling Window (30s)')
pyplot.ylabel('Average CPU Utilisation (x 100)')
pyplot.title('MatMul/Training')
pyplot.legend(['PPO', 'RPPO', 'DRQN'], loc='upper right')
pyplot.show()


# Memory Utilization
ppox = df_ppo_['step'].to_numpy()
ppoy = df_ppo_['mem'].to_numpy()
ppoy = df_ppo_['mem'].rolling(window=500, min_periods=10).mean().to_numpy()
rppox = df_rppo_['step'].to_numpy()
rppoy = df_rppo_['mem'].to_numpy()
rppoy = df_rppo_['mem'].rolling(window=500, min_periods=10).mean().to_numpy()
dqn_x = df_dqn_['step'].to_numpy()
dqn_y = df_dqn_['mem'].to_numpy()
dqn_y = df_dqn_['mem'].rolling(window=500, min_periods=10).mean().to_numpy()
pyplot.plot(ppox, ppoy, color='red')
pyplot.plot(rppox[:7705], rppoy[:7705], color='green')
pyplot.plot(dqn_x[:7170], dqn_y[:7170], color='blue')
pyplot.xlabel('Sampling Window (30s)')
pyplot.ylabel('Average Memory Utilisation (x 100)')
pyplot.title('MatMul/Training')
pyplot.legend(['PPO', 'RPPO', 'DRQN'], loc='upper right')
pyplot.show()

# Average Execution time
ppox = df_ppo_['step'].to_numpy()
ppoy = df_ppo_['avg_execution_time'].to_numpy()
ppoy = df_ppo_['avg_execution_time'].rolling(window=200, min_periods=1).mean().to_numpy()
rppox = df_rppo_['step'].to_numpy()
rppoy = df_rppo_['avg_execution_time'].to_numpy()
rppoy = df_rppo_['avg_execution_time'].rolling(window=200, min_periods=1).mean().to_numpy()
dqn_x = df_dqn_['step'].to_numpy()
dqn_y = df_dqn_['avg_execution_time'].to_numpy()
dqn_y = df_dqn_['avg_execution_time'].rolling(window=200, min_periods=1).mean().to_numpy()
pyplot.plot(ppox, ppoy, color='red')
pyplot.plot(rppox[:7705], rppoy[:7705], color='green')
pyplot.plot(dqn_x[:7170], dqn_y[:7170], color='blue')
pyplot.xlabel('Sampling Window (30s)')
pyplot.ylabel('Average Execution Time (s)')
pyplot.title('MatMul/Training')
pyplot.legend(['PPO', 'RPPO', 'DRQN'], loc='lower right')
pyplot.show()

In [ ]:
# Throughput
ppox = df_ppo_['step'].to_numpy()
ppoy = df_ppo_['throughput'].to_numpy()
ppoy = df_ppo_['throughput'].rolling(window=500, min_periods=10).mean().to_numpy()
rppox = df_rppo_['step'].to_numpy()
rppoy = df_rppo_['throughput'].to_numpy()
rppoy = df_rppo_['throughput'].rolling(window=500, min_periods=10).mean().to_numpy()
dqn_x = df_dqn_['step'].to_numpy()
dqn_y = df_dqn_['throughput'].to_numpy()
dqn_y = df_dqn_['throughput'].rolling(window=500, min_periods=10).mean().to_numpy()
pyplot.plot(ppox, ppoy, color='red')
pyplot.plot(rppox[:7705], rppoy[:7705], color='green')
pyplot.plot(dqn_x[:7170], dqn_y[:7170], color='blue')
pyplot.xlabel('Sampling Window (30s)')
pyplot.ylabel('Throughput % (Successful/Total Requests)')
pyplot.title('MatMul/Training')
pyplot.legend(['PPO', 'RPPO', 'DRQN'], loc='lower right')
pyplot.show()

In [ ]:
# Scatter plots Execution time vs Throughput
import matplotlib.pyplot as plt

# data
throughput = [i for i in ppoy if i <= 100]
response_time = df_ppo_['avg_execution_time'][:len(throughput)]

# Scatter Plot
plt.scatter(throughput, response_time)
plt.xlabel('Throughput')
plt.ylabel('Execution Time')
plt.title('Throughput vs. Execution Time (Scatter Plot)')
plt.grid()
plt.show()

# data
throughput = rppoy
response_time = df_rppo_['avg_execution_time']

# Scatter Plot
plt.scatter(throughput, response_time)
plt.xlabel('Throughput')
plt.ylabel('Execution Time')
plt.title('Throughput vs. Execution Time (Scatter Plot)')
plt.grid()
plt.show()



# data
throughput = dqn_y[:7170]
response_time = df_dqn_['avg_execution_time'][:7170]

# Scatter Plot
plt.scatter(throughput, response_time)
plt.xlabel('Throughput')
plt.ylabel('Execution Time')
plt.title('Throughput vs. Execution Time (Scatter Plot)')
plt.grid()
plt.show()

## LSTM-PPO, PPO, DRQN Evaluation Data

In [ ]:
# evaluation data ------- environment scalars
log_dir = './${PPO_EVAL_DIR}/scalars'
reader = SummaryReader(log_dir, pivot=True)
# print(reader.get_tags())
df_ppo_ = reader.tensors


log_dir = './${RPPO_EVAL_DIR}/scalars'
reader = SummaryReader(log_dir, pivot=True)
# print(reader.get_tags())
df_rppo_ = reader.tensors

log_dir = './${DQN_EVAL_DIR}'
reader = SummaryReader(log_dir, pivot=True)
# print(reader.get_tags())
df_dqn_ = reader.tensors

In [ ]:
import matplotlib.pyplot as plt
ppox = df_ppo_['step'].to_numpy()
ppoy = df_ppo_['throughput'].to_numpy()
# ppoy = df_ppo_['throughput'].rolling(window=100, min_periods=1).mean()
rppox = df_rppo_['step'].to_numpy()
rppoy = df_rppo_['throughput'].to_numpy()
# rppoy = df_rppo_['throughput'].rolling(window=100, min_periods=1).mean()
dqnx = df_dqn_['step'].to_numpy()
dqny = df_dqn_['throughput'].to_numpy()
# rppoy = df_rppo_['throughput'].rolling(window=100, min_periods=1).mean()


# PPO Scatter Plot
plt.scatter(df_ppo_['throughput'].to_numpy(), df_ppo_['avg_execution_time'].to_numpy())
plt.xlabel('Throughput')
plt.ylabel('Execution Time')
plt.title('Throughput vs. Execution Time (Scatter Plot)')
plt.grid()
plt.show()

# Rppo data
throughput = rppoy
response_time = df_rppo_['avg_execution_time']
# Scatter Plot
plt.scatter(df_rppo_['throughput'].to_numpy(), df_rppo_['avg_execution_time'].to_numpy())
plt.xlabel('Throughput')
plt.ylabel('Execution Time')
plt.title('Throughput vs. Execution Time (Scatter Plot)')
plt.grid()
plt.show()

# DQRN data
throughput = dqny
response_time = df_dqn_['avg_execution_time'].to_numpy()
# print(len(throughput), len(response_time))
# Scatter Plot
plt.scatter(df_dqn_['throughput'].to_numpy()[:200], df_dqn_['avg_execution_time'].to_numpy()[:200])
plt.xlabel('Throughput')
plt.ylabel('Execution Time')
plt.title('Throughput vs. Execution Time (Scatter Plot)')
plt.grid()
plt.show()

# Results comparison
ppo_not = [i for i in df_ppo_['throughput'].to_numpy()[:200] if i < 100 and i != 0]
rppo_not = [i for i in df_rppo_['throughput'] if i < 100 and i != 0]
dqn_not = [i for i in df_dqn_['throughput'].to_numpy()[:200] if i < 100 and i != 0]
print(len(ppo_not), len(df_ppo_['throughput'].to_numpy()[:200]), len(rppo_not), len(df_rppo_['throughput']))
print(f'rppo better results tha ppo by {(len(rppo_not)/len(ppo_not))*100}')
print(f'rppo better results tha dqn by {(len(rppo_not)/len(dqn_not))*100}')

# Replicas comparison
ppox = df_ppo_['step'].to_numpy()
ppoy = df_ppo_['replicas'].to_numpy()
# ppoy = df_ppo_['throughput'].rolling(window=100, min_periods=1).mean()
rppox = df_rppo_['step'].to_numpy()
rppoy = df_rppo_['replicas'].to_numpy()
# rppoy = df_rppo_['throughput'].rolling(window=100, min_periods=1).mean()
dqnx = df_dqn_['step'].to_numpy()
dqny = df_dqn_['replicas'].to_numpy()
# rppoy = df_rppo_['throughput'].rolling(window=100, min_periods=1).mean()

pyplot.plot([i for i in range(len(ppoy))][:200], ppoy[:200], color='red')
pyplot.plot([i for i in range(len(rppoy))][:200], rppoy[:200], color='green')
pyplot.plot([i for i in range(len(dqny))][:200], dqny[:200], color='blue')
pyplot.xlabel('Sampling Window (30s)')
pyplot.ylabel('Replicas (# of Functions)')
pyplot.title('MatMul/Evaluation')
pyplot.legend(['PPO', 'RPPO', 'DRQN'], loc='upper right')
pyplot.show()

In [ ]:

# Plot replica comparison
_data = {'PPO': [min(ppoy[:200]),max(ppoy[:200]),mean(ppoy[:200]), df_ppo_['avg_execution_time'][:200].mean()],
        'RPPO': [min(rppoy[:200]),max(rppoy[:200]),mean(rppoy[:200]), df_rppo_['avg_execution_time'][:200].mean()],
        'DRQN': [min(dqny[:200]),max(dqny[:200]),mean(dqny[:200]), df_dqn_['avg_execution_time'][:200].mean()]}
_df = pd.DataFrame(_data,columns=['PPO', 'RPPO', 'DRQN'], index = ['Min. Replicas','Max. Replicas','Mean Replicas', 'Execution Time(s) '])

# Multiple bar chart
_df.plot.bar()

# Display the plot
pyplot.show()

In [ ]:
# Plot throughput sample comparison
X = ['Total Samples','100% Throughput','Non-Zero Throughput', 'Mean Throughput %']
_data = {'PPO': [200,115,61, 67],
        'RPPO': [200,173,11, 85],
        'DRQN': [200, 146, 50, 67]}
_df = pd.DataFrame(_data,columns=['PPO', 'RPPO', 'DRQN'], index = ['Total Samples','100% Throughput','Non-Zero Throughput', 'Mean Throughput %'])

# Multiple bar chart
_df.plot.bar()

# Display the plot
pyplot.show()

## HPA and RPS Plots

In [ ]:
# Collection of HPA and RPS data in a similar fashion
# training data ------- environment scalars
log_dir = './${HPA_LOG_DIR}'
reader = SummaryReader(log_dir, pivot=True)
# print(reader.get_tags())
df_hpa = reader.tensors


log_dir = './${RPS_LOG_DIR}'
reader = SummaryReader(log_dir, pivot=True)
# print(reader.get_tags())
df_rps = reader.tensors

In [ ]:
hpax = df_hpa['step'].to_numpy()
hpay = df_hpa['throughput'].to_numpy()
hpay = df_hpa['throughput'].rolling(window=10, min_periods=10).mean().to_numpy()
rpsx = df_rps['step'].to_numpy()
rpsy = df_rps['throughput'].to_numpy()
rpsy = df_rps['throughput'].rolling(window=100, min_periods=10).mean().to_numpy()
pyplot.plot(hpax, hpay, color='red')
pyplot.plot(rpsx, rpsy, color='green')
pyplot.xlabel('Sampling Window (30s)')
pyplot.ylabel('Throughput % (Successful/Total Requests)')
pyplot.title('MatMul/HPA_RPS')
pyplot.legend(['HPA', 'RPS'], loc='lower right')
pyplot.show()

# data
throughput = [i for i in hpay if i <= 100]
response_time = df_ppo_['avg_execution_time'].to_numpy()[:len(throughput)]


hpa_not = [i for i in df_hpa['throughput'].to_numpy()[:200] if i < 100 and i != 0]
rps_not = [i for i in df_rps['throughput'] if i < 100 and i != 0]

print(len(hpa_not), len(df_hpa['throughput'].to_numpy()[:200]), len(rps_not), len(df_rps['throughput']))
print(f'better results by {(len(rps_not)/len(hpa_not))*100}')
print(len([i for i in df_hpa['throughput'][:200] if i == 100]))
print(len([i for i in df_rps['throughput'][:200] if i == 100]))

# Scatter Plot
plt.scatter(df_hpa['throughput'], df_hpa['avg_execution_time'])
plt.xlabel('Throughput')
plt.ylabel('Execution Time')
plt.title('Throughput vs. Execution Time (Scatter Plot)')
plt.grid()
plt.show()

# Scatter Plot
plt.scatter(df_rps['throughput'], df_rps['avg_execution_time'])
plt.xlabel('Throughput')
plt.ylabel('Execution Time')
plt.title('Throughput vs. Execution Time (Scatter Plot)')
plt.grid()
plt.show()

hpax = df_hpa['step'].to_numpy()
hpay = df_hpa['replicas'].to_numpy()
# hpay = df_hpa['throughput'].rolling(window=10, min_periods=1).mean()
rpsx = df_rps['step'].to_numpy()
rpsy = df_rps['replicas'].to_numpy()
# rpsy = df_rps['throughput'].rolling(window=10, min_periods=1).mean()
pyplot.plot(hpax, hpay, color='red')
pyplot.plot(rpsx, rpsy, color='green')
pyplot.xlabel('Sampling Window (30s)')
pyplot.ylabel('Replicas')
pyplot.title('MatMul/HPA_RPS')
pyplot.legend(['HPA', 'RPS'], loc='upper right')
pyplot.show()

### Final Results comparison between LSTM-PPO, PPO, DRQN, HPA and RPS

In [ ]:
ppo_not = [i for i in df_ppo_['throughput'].to_numpy()[:200] if i < 100 and i != 0]
rppo_not = [i for i in df_rppo_['throughput'].to_numpy()[:200] if i < 100 and i != 0]
dqn_not = [i for i in df_dqn_['throughput'].to_numpy()[:200] if i < 100 and i != 0]

print(len(ppo_not), len(df_ppo_['throughput'].to_numpy()[:200]), len(rppo_not), len(df_rppo_['throughput'].to_numpy()[:200]), len(dqn_not), len(df_dqn_['throughput'].to_numpy()[:200]))
print(f'rppo better results than ppo by {(len(rppo_not)/len(ppo_not))*100}')
print(f'rppo better results than dqn by {(len(rppo_not)/len(dqn_not))*100}')
print(len([i for i in df_ppo_['throughput'].to_numpy()[:200] if i == 100]))
print(len([i for i in df_rppo_['throughput'].to_numpy()[:200] if i == 100]))
print(len([i for i in df_dqn_['throughput'].to_numpy()[:200] if i == 100]))


print(mean(ppo_not))
print(mean(rppo_not))
print(mean(dqn_not))

print(df_ppo_['avg_execution_time'].to_numpy()[:200].mean())
print(df_rppo_['avg_execution_time'].to_numpy()[:200].mean())
print(df_dqn_['avg_execution_time'].to_numpy()[:200].mean())
print(mean(ppoy[:200]))
print(mean(rppoy[:200]))
print(mean(dqny[:200]))
print(min(ppoy[:200]))
print(min(ppoy[:200]))
print(min(dqny[:200]))

d = df_ppo_['replicas'].to_numpy()[:200] - df_rppo_['replicas'].to_numpy()[:200]
print(d.mean())

d = df_dqn_['replicas'].to_numpy()[:200] - df_rppo_['replicas'].to_numpy()[:200]
print(d.mean())

ppo_not = [i for i in df_hpa['throughput'].to_numpy()[:200] if i < 100 and i != 0]
rppo_not = [i for i in df_rps['throughput'] if i < 100 and i != 0]

print(len(ppo_not), len(df_hpa['throughput'].to_numpy()[:200]), len(rppo_not), len(df_rps['throughput']))
print(f'better results by {(len(rppo_not)/len(ppo_not))*100}')
print(len([i for i in df_hpa['throughput'].to_numpy()[:200] if i == 100]))
print(len([i for i in df_rps['throughput'].to_numpy()[:200] if i == 100]))